In [1]:
import threading

# Imports used in Selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from time import sleep

from bs4 import BeautifulSoup

# general python imports i.e. working with dataframes and basic maths
import pandas as pd
import numpy as np

import math
import datetime as datetime
from tqdm import tqdm_notebook as tqdm
import ast as ast

#dashboard imports
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table

# used to get most common words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, precision_score
pd.set_option('display.max_columns', None)

# Import some text packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

#folium doesn't seem like it works in dash, but really good code anyway
import folium
import os
import requests



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Joey\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [51]:
pwd

'C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash'

# Entering Postcodes and getting restaurant URLs

In [2]:
#should cover all of greater london
postcode_list=['E8 1EX', 'E17 4PP', 'E6 6AB', 'SE7 8BL', 'SE2 0XT',
              'SW16 1BB', 'SW4 8EL', 'SW17 9NA', 'SW19 5AE', 'SW6 3PR', 
               'W5 5TH','NW8 9NH', 'NW6 1RZ','NW7 1NB','N8 8JD', 
               'E16 1XL', 'SE25 6AB', 'SE23 1NW', 'SE1 9SG','N21 1LE',
              'SE10 8JQ','EC1V 4NB','SW1V 2BP','W6 7NL','TW8 8JF','TW9 1DN',
              'BR1 3PX','N1 8AB']

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
#driver = webdriver.Chrome(executable_path='C:\\webdriver\\chromedriver.exe',chrome_options=options)

def shop_grabber2(postcode_list):
    driver = webdriver.Chrome(executable_path='C:\\webdriver\\chromedriver.exe',chrome_options=options)
    
    for i in tqdm(postcode_list):
        
        driver.get('https://www.just-eat.co.uk/')

        post_field = driver.find_element_by_class_name("Form_input_y1xkO")
        post_field.clear()
        #post_field.click()
        post_field.send_keys(i)
        find_retaurants=driver.find_element_by_class_name("Form_btn_TuG34")
        find_retaurants.click()

        

        for shop in driver.find_elements_by_class_name('c-listing-item-link'):
            url=shop.get_attribute('href')
            
            try:
                cusine=shop.find_element_by_class_name('c-listing-item-text').text
            except:
                cusine=None
                
            food_list2.append([url,cusine])

    
    return 

In [6]:
food_list2=[]


post1=postcode_list[0:14]
post2=postcode_list[14:28]
#post3=postcode_list[20:28]
# post4=postcode_list[21:28]


jobs = []


thread1 = threading.Thread(target=shop_grabber2, args=(post1,), )
thread2 = threading.Thread(target=shop_grabber2, args=(post2,), )
#thread3 = threading.Thread(target=shop_grabber2, args=(post3,), )
# thread4 = threading.Thread(target=shop_grabber2, args=(post4,), )
jobs.append(thread1)
jobs.append(thread2)
#jobs.append(thread3)
# jobs.append(thread4)

# Start the threads
for j in jobs:
    j.start()
    
# Ensure all of the threads have finished
for j in jobs:
    j.join()
    
len(food_list2)   

C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

use options instead of chrome_options



6245

In [16]:
url_df=pd.DataFrame(food_list2,columns=['url','cuisine']).drop_duplicates(['url'],keep='first')
url_df.to_csv('menu_urls.csv')

print('just eat restaurants in greater london=',len(url_df))

just eat restaurants in greater london= 3732


In [15]:
url_df=pd.read_csv('C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash\\menu_urls.csv',index_col=0)

# Getting each restaurant's postcode, name and URL for review page - potential to scrape menu?

In [10]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

shops=[]

def shopgrabber3(restaurants):
    driver = webdriver.Chrome(executable_path='C:\\webdriver\\chromedriver.exe',chrome_options=options)
    
    for i in tqdm(restaurants):
        
        driver.get(i)
        
        link_2_reviews=i.replace('/menu','/reviews')
        
        try:
            name=driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[1]/div[2]/h1').text
        except:
            try:
                name=driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[2]/div[2]/h1').text
            except:
                name=np.nan
                #print('bullshit with name')
                
        try:
            postcode=driver.find_element_by_xpath('//*[@id="postcode"]').text
        except:
            postcode=np.nan
            #print('bullshit with postcode')    
        
       
        
        shops.append([name,postcode,link_2_reviews,i])
        
    driver.quit()
        
    return

In [192]:
len(url_df)

3732

In [193]:
shops=[]

list1=url_df.url[0:1866]
list2=url_df.url[1866:3732]

jobs = []

thread1 = threading.Thread(target=shopgrabber3, args=(list1,), )
thread2 = threading.Thread(target=shopgrabber3, args=(list2,), )

jobs.append(thread1)
jobs.append(thread2)

# Start the threads
for j in jobs:
    j.start()
    
# Ensure all of the threads have finished
for j in jobs:
    j.join()
    
len(shops)   

3732

In [194]:
df=pd.DataFrame(shops,columns=['name','postcode','link_2_reviews','url']).merge(url_df,how='inner')
df.to_csv('restaurants')

# Run From Here When Refreshing

In [2]:
df=pd.read_csv('restaurants',index_col=0)

In [3]:
df['pc_prefix']=[str(x).split(' ')[0] for x in df.postcode]
df.groupby(['pc_prefix']).count().sort_values('postcode',ascending=False)['postcode'].head(20)

pc_prefix
N1      88
SE1     80
E17     80
E1      78
E2      75
SW17    68
NW6     68
CR0     65
SW11    60
SW19    60
SW16    56
E14     53
BR1     48
W2      48
SW18    46
W12     45
E13     45
NW10    44
SE18    44
E8      44
Name: postcode, dtype: int64

In [4]:
len(df)

3732

# Scraping Ratings and Reviews

In [11]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

def review_getter(ziplist):    
    
    driver = webdriver.Chrome(executable_path='C:\\webdriver\\chromedriver.exe',chrome_options=options)
    
    for i in tqdm(ziplist):
        
        driver.get(i[0])
        try:
            overall_rating=[]
            total_reviews=[]

            try:
                overall_rating=float(driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[2]/div[2]/span/span').text)
                overall_rating=round((overall_rating/6)*100,1)
                total_reviews=int(driver.find_element_by_xpath('//*[@id="restaurant"]/div[2]/div[1]/div[2]/div/p/a').text.split(' ')[0])
            except:
                overall_rating=np.nan
                total_reviews=np.nan


            dates=[]
            stars=[]
            reviews=[]

            #try:
            for review in driver.find_element_by_class_name('restaurantRatings').find_elements_by_css_selector('li'):

                #date
                try:
                    date=review.find_element_by_class_name('date').text
                    date2=datetime.datetime.strptime(date,"%d/%m/%Y").date()
                    dates.append(date2)
                except:
                    dates.append(np.nan)

                #rating
                try:
                    rating=float(review.find_elements_by_css_selector('img')[0].get_attribute('title').split(' ')[0])
                    rating2=round((rating/6)*100,1)
                    stars.append(float(rating2))
                except:
                    stars.append(np.nan)

                #review
                try:
                    #comment=review.text.split('\n')[2]
                    comment=review.find_element_by_class_name('comments').text
                    reviews.append(comment)
                except:
                    reviews.append(None)
    #         except:
    #             dates=np.nan
    #             stars=np.nan
    #             reviews=np.nan

            review_dict = {'overall_rating':overall_rating,
                           'total_reviews':total_reviews,
                            'date' : dates,
                           'stars' : stars,
                           'review_text' : reviews,
                           'link_2_reviews' : i[0],
                           'review_target': i[1]}


            #reviews_df = pd.DataFrame(reviews_list)

            review_df=pd.DataFrame(review_dict)

            all_reviews.append(review_df)
        except:
            pass
        
    driver.quit()
    
    return 

933.0

In [12]:
ziplist=list(zip(df.link_2_reviews,df.name))

all_reviews=[]

threads=4
x=int(round(len(df)/threads))
remainder=int(((len(df)/threads)-x)*threads)

list1=ziplist[0:x]
list2=ziplist[x:(2*x)]
list3=ziplist[(2*x):(3*x)]
list4=ziplist[(3*x):(4*x)+remainder]


jobs = []

thread1 = threading.Thread(target=review_getter, args=(list1,), )
thread2 = threading.Thread(target=review_getter, args=(list2,), )
thread3 = threading.Thread(target=review_getter, args=(list3,), )
thread4 = threading.Thread(target=review_getter, args=(list4,), )


jobs.append(thread1)
jobs.append(thread2)
jobs.append(thread3)
jobs.append(thread4)

# Start the threads
for j in jobs:
    j.start()
    
# Ensure all of the threads have finished
for j in jobs:
    j.join()
    
len(all_reviews)   

C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning:

use options instead of chrome_options



0

In [70]:
jobs=[]

def jobber(thread_count,dataframe):

    threads=thread_count
    x=int(round(len(dataframe)/threads))
    remainder=int(((len(dataframe)/threads)-x)*thread_count)
    
    for i in range(threads-1):
        jobs.append(threading.Thread(target=review_getter, args=(ziplist[(i-1)*x:(i*x)])))
        
    #jobs.append(threading.Thread(target=review_getter, args=(ziplist[((thread_count-1)*x):(thread_count*x)+remainder])))
        
    return 

jobber(4,df)

# Start the threads
for j in jobs:
    j.start()
    
# Ensure all of the threads have finished
for j in jobs:
    j.join()

Exception in thread Thread-44:
Traceback (most recent call last):
  File "C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
TypeError: review_getter() missing 1 required positional argument: 'ziplist'
Exception in thread Thread-45:
Traceback (most recent call last):
  File "C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
TypeError: review_getter() takes 1 positional argument but 933 were given
Exception in thread Thread-46:
Traceback (most recent call last):
  File "C:\Users\Joey\AppData\Local\Continuum\anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\User

In [14]:
#can apply NLP to test 
test=pd.concat(all_reviews).reset_index().drop(columns=['index'])
test['date_range']=test.date

test2=test.groupby('link_2_reviews').aggregate({'overall_rating':'max',
                                               'total_reviews':'max',
                                               'date':'max',
                                               'date_range': lambda x: max(x) - min(x),
                                               'stars':'mean',
                                               'review_text':'count',
                                               'review_target':'first'}).reset_index()

df2=test2.merge(df)

In [15]:
df2=df2.rename(columns={'date':'most_recent_review','review_text':'reviews','stars':'trending_score'})
df2.trending_score=df2.trending_score.apply(lambda x: round(x,1))

In [43]:
df3=df2[(df2.total_reviews>50) &
        (df2.most_recent_review>datetime.date(2019, 3, 1))]

len(df3)

2353

In [18]:
list_lists=[x.split(' ') for x in df2.cuisine]
flat_list = [item for sublist in list_lists for item in sublist]

from collections import Counter

Counter(flat_list)


Counter({'Burgers': 318,
         'Fast': 30,
         'Food': 34,
         'Halal': 1458,
         'African': 50,
         'Nigerian': 18,
         'Caribbean': 125,
         'English': 24,
         'Pizza': 745,
         'Italian': 415,
         'Gourmet': 28,
         'Breakfast': 62,
         'Grill': 174,
         'Fish': 105,
         '&': 105,
         'Chips': 105,
         'Korean': 10,
         'Asian': 37,
         'Kebab': 299,
         'Indian': 740,
         'Fusion': 8,
         'Sushi': 165,
         'Chinese': 453,
         'Turkish': 129,
         'Milkshakes': 9,
         'American': 207,
         'Oriental': 260,
         'Lebanese': 110,
         'Middle': 34,
         'Eastern': 38,
         'Curry': 430,
         'Thai': 171,
         'South': 25,
         'Jamaican': 64,
         'Desserts': 81,
         'Dim': 11,
         'Sum': 11,
         'Seafood': 7,
         'Bangladeshi': 64,
         'Drinks': 14,
         'Peri': 238,
         'Chicken': 359,
        

In [46]:
#Additional data prep

df3=df3.dropna()

chains=['kfc' , 'burger king' , 'subway' , 'pizza hut' , 'papa j']

df3['corporation']=['chain' if any(thing in x.lower() for thing in chains) else 'independent' for x in df3.name]
      
df3[df3.total_reviews>50].to_csv('test.csv')

postcodes=[]
for thing in df3.pc_prefix:
    if 'BR' in thing:
        a='BR'
    elif 'DA' in thing:
        a='DA'
    elif 'HA' in thing:
        a='HA'
    elif 'EN' in thing:
        a='EN'
    elif 'IG' in thing:
        a='IG'
    elif 'KT' in thing:
        a='KT'
    elif 'SM' in thing:
        a='SM'
    elif 'TW' in thing:
        a='TW' 
    elif 'UB' in thing:
        a='UB'
    elif 'WC1' in thing:
        a='WC1'    
    elif 'WC2' in thing:
        a='WC2'
    elif 'EC1' in thing:
        a='EC1'        
    elif 'EC2' in thing:
        a='EC2'        
    elif 'EC3' in thing:
        a='EC3'        
    elif 'EC4' in thing:
        a='EC4' 
    else:
        a=thing
    postcodes.append(a)
        

df3.pc_prefix=postcodes 

In [47]:
strata=pd.read_csv('C:\\Users\\Joey\\Desktop\\uswitch postcode data\\190122 Postcode Level - MASTER.csv')

df3['postcode_clean']=df3.postcode.apply(lambda x: x.replace(' ',''))
df3=df3.merge(strata[['postcode_clean','latitude','longitude']],how='left')
df3.to_csv('takeaway_data.csv')

## Dash Code - Run in Anaconda Prompt for preview

In [ ]:
##using bootstrap

import numpy as np
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py
import base64

mapbox_access_token = 'pk.eyJ1Ijoiam9leTkxIiwiYSI6ImNqdDA3emFzazA5Y2IzeW56ZHRtN3Q3NG0ifQ.0l4idLnp8xRT8Y_Ipyj7vg'

#dataframe for maps and table
table_df = pd.read_csv('C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash\\takeaway_data.csv',index_col=0)
table_df=table_df.rename(columns={'total_reviews':'Reviews ','trending_score':'Trending Score ','overall_rating':'Lifetime Score '})

table_df = table_df.sort_values('Trending Score ',ascending=False)
table_df=table_df.dropna()

zipped=zip(table_df.name
           ,table_df.postcode
           ,table_df['Lifetime Score '])

table_df['text']=[x +', '+y+', Rating: '+str(z) for x,y,z in zipped]

#for cuisine selector
acceptable=['Chicken','Pizza','Chinese','Italian','Caribbean','Burgers','Kebab','Sandwiches','British','Thai',
           'Vietnamese','Japanese','African','Indian','Curry','Mexican','Mediterranean']


#for bar chart 
acceptable2=['Chicken','Pizza','Burgers','Sandwiches','Chinese','Indian','Kebab','British','Italian','Caribbean','Thai',
           'Vietnamese','Japanese']

cuisine=[]
average_rating=[]
volume=[]
restaurant_type=[]

    
#Human Theory Logo
image_filename = 'C:\\Users\\Joey\\Google Drive\\5. Python Programs\\Webscraping\\2.Unfinished\\Chicken Dash\\Human-Theory-Typemark-Sunset-Orange_RGB.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#top of site navigation bar

navbar = dbc.NavbarSimple(
    html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),height='70', 
             width='140')
        ], href='https://www.humantheory.co.uk'),
    
#     brand="Human Theory",
#     brand_href="https://www.humantheory.co.uk/",
    sticky="top",
)


#2 containers in one row

body = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H2("Best in your area"),
                        dcc.Dropdown(id='postSelector',
                                     options=[{'label': i, 'value': i} for i in sorted(table_df.pc_prefix.unique())],
                                     value='E8',
                                     style={'width': '90%'}),
                        
                       dcc.Dropdown(id='cuisineSelector2',
                                    options=[{'label': i, 'value': i} for i in sorted(acceptable)],
                                    value='Chicken',
                                    style={'width': '90%'}),
                        
                        html.Div(id='table-container')
                    ],
                    md=4,
                ),
                dbc.Col(
                    [
                       html.H2("Takeaway Map of London"),
                        
                       dcc.Dropdown(id='cuisineSelector',
                                    options=[{'label': i, 'value': i} for i in sorted(acceptable)],
                                    value='Chicken',
                                    style={'width': '48%'}),
                        
                       dcc.RadioItems(id='corpSelector',
                                      options=[{'label': i, 'value': i} for i in ['chain', 'independent']],
                                      value='independent',
                                      labelStyle={'display': 'inline-block'}), 
                        
                      dcc.Graph(id='area-shops')
                    ]
                ),
            ]
        )
    ],
    className="mt-4",
)

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])

app.layout = html.Div([navbar, body])





#callbacks

def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
        html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(len(dataframe))]
    )

@app.callback(
    dash.dependencies.Output('table-container', 'children'),
    [dash.dependencies.Input('postSelector', 'value'),
     dash.dependencies.Input('cuisineSelector2', 'value')])

def display_table(postSelector,cuisineSelector2):

    #df = table_df[(table_df.pc_prefix==postSelector) ][['name',  'Trending Score ', 'Lifetime Score ','Reviews ']].head(10)
    
    df = table_df[(table_df.pc_prefix==postSelector) & 
                  (table_df.cuisine.str.contains(cuisineSelector2, regex=False))][['name',  'Trending Score ', 'Lifetime Score ','Reviews ']].head(10)

    return generate_table(df)



#map callback

@app.callback(
    dash.dependencies.Output('area-shops', 'figure'),
    [#dash.dependencies.Input('postSelector', 'value'),
    dash.dependencies.Input('cuisineSelector', 'value'),
    dash.dependencies.Input('corpSelector', 'value')
    ])

def update_graph(cuisineSelector='Chicken',corpSelector='independent'):

    
    df=table_df[#(table_df.pc_prefix==postSelector) & 
                (table_df.cuisine.str.contains(cuisineSelector, regex=False)) 
                & (table_df.corporation==corpSelector)
    ]

    #red points
    data = [
        go.Scattermapbox(
            lat=df[df['Lifetime Score ']<=60]['latitude'],
            lon=df[df['Lifetime Score ']<=60]['longitude'],
            text = df[df['Lifetime Score ']<=60]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'red',opacity = 1),
            name='shit food',
            showlegend=False
        ),

    #light red
        go.Scattermapbox(
            lat=df[(df['Lifetime Score ']>60) & (df['Lifetime Score ']<=70)]['latitude'],
            lon=df[(df['Lifetime Score ']>60) & (df['Lifetime Score ']<=70)]['longitude'],
            text = df[(df['Lifetime Score ']>60) & (df['Lifetime Score ']<=70)]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'lightsalmon',opacity = 1),
            name='bad food',
            showlegend=False),


    #lawn green

        go.Scattermapbox(
            lat=df[(df['Lifetime Score ']>70) & (df['Lifetime Score ']<=80)]['latitude'],
            lon=df[(df['Lifetime Score ']>70) & (df['Lifetime Score ']<=80)]['longitude'],
            text = df[(df['Lifetime Score ']>70) & (df['Lifetime Score ']<=80)]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'lawngreen',opacity = 1),
            name='good food',
            showlegend=False),

    #green

        go.Scattermapbox(
            lat=df[df['Lifetime Score ']>80]['latitude'],
            lon=df[df['Lifetime Score ']>80]['longitude'],
            text = df[df['Lifetime Score ']>80]['text'],
            mode='markers',
            marker=dict(size= 13,color = 'green',opacity = 1),
            name='excellent food',
            showlegend=False),
    ]

    layout = go.Layout(
        autosize=False,
        width=700,
        height=450,
        hovermode='closest',
        margin=go.layout.Margin(
        l=1,
        r=1,
         b=20,
         t=1,
#         pad=4
    ),
        mapbox=go.layout.Mapbox(accesstoken=mapbox_access_token,
                                bearing=0,
                                center=go.layout.mapbox.Center(lat=51.512324, lon=-0.075366),
                                pitch=0,
                                zoom=10),
    )

    return go.Figure(data=data, layout=layout)



if __name__ == '__main__':
    app.run_server(debug=True)